In [38]:
import numpy as np
import pandas as pd
import math
import os.path
import json
import time
import datetime

In [19]:
DATA_ROOT_PATH = '/src/data/mi-voting'

MI_VOTER_FILENAME = 'EntireStateVotersOct2020.csv'
MI_VOTER_HISTORY_FILENAME = 'EntireStateVoterHistoryOct2020.csv'

In [6]:
voter_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, MI_VOTER_FILENAME), encoding = "ISO-8859-1")
voter_hist_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, MI_VOTER_HISTORY_FILENAME), encoding = "ISO-8859-1")
voter_hist_df['ELECTION_DATE'] = pd.to_datetime(voter_hist_df['ELECTION_DATE'] )

In [11]:
voter_df.head(3)

,LAST_NAME,FIRST_NAME,MIDDLE_NAME,NAME_SUFFIX,YEAR_OF_BIRTH,GENDER,REGISTRATION_DATE,STREET_NUMBER_PREFIX,STREET_NUMBER,STREET_NUMBER_SUFFIX,...,COUNTY_COMMISSIONER_DISTRICT_CODE,COUNTY_COMMISSIONER_DISTRICT_NAME,VILLAGE_DISTRICT_CODE,VILLAGE_DISTRICT_NAME,VILLAGE_PRECINCT,SCHOOL_PRECINCT,IS_PERMANENT_ABSENTEE_VOTER,VOTER_STATUS_TYPE_CODE,UOCAVA_STATUS_CODE,UOCAVA_STATUS_NAME
0,NYKAMP,SALLY,ANN,NaN,1959,F,2006-10-12,NaN,38.0,NaN,...,7004,4th District,NaN,NaN,NaN,NaN,N,A,N,NON-UOCAVA
1,BERTOLINI,CHRISTOPHER,PAUL,NaN,1979,M,2020-09-14,NaN,42330.0,NaN,...,5005,5th District,NaN,NaN,NaN,NaN,Y,A,N,NON-UOCAVA
2,MCKAY,RASHAWN,NaN,NaN,1998,M,2018-12-13,NaN,3617.0,NaN,...,3903,3rd District,NaN,NaN,NaN,NaN,N,A,N,NON-UOCAVA


In [12]:
voter_df.dtypes

LAST_NAME                             object
FIRST_NAME                            object
MIDDLE_NAME                           object
NAME_SUFFIX                           object
YEAR_OF_BIRTH                          int64
GENDER                                object
REGISTRATION_DATE                     object
STREET_NUMBER_PREFIX                  object
STREET_NUMBER                        float64
STREET_NUMBER_SUFFIX                  object
DIRECTION_PREFIX                      object
STREET_NAME                           object
STREET_TYPE                           object
DIRECTION_SUFFIX                      object
EXTENSION                             object
CITY                                  object
STATE                                 object
ZIP_CODE                               int64
MAILING_ADDRESS_LINE_ONE              object
MAILING_ADDRESS_LINE_TWO              object
MAILING_ADDRESS_LINE_THREE            object
MAILING_ADDRESS_LINE_FOUR             object
MAILING_AD

In [28]:
age_cond=100
YEARS= 2020 - age_cond
mi_voter_senior_df = voter_df[voter_df['YEAR_OF_BIRTH']<YEARS]

In [29]:
len(mi_voter_senior_df.index)

12530

In [36]:
voter_hist_df.dtypes

VOTER_IDENTIFICATION_NUMBER             int64
COUNTY_CODE                             int64
COUNTY_NAME                            object
JURISDICTION_CODE                       int64
JURISDICTION_NAME                      object
SCHOOL_DISTRICT_CODE                    int64
SCHOOL_DISTRICT_NAME                   object
ELECTION_DATE                  datetime64[ns]
IS_ABSENTEE_VOTER                      object
dtype: object

In [35]:
voter_hist_df.head(5)

,VOTER_IDENTIFICATION_NUMBER,COUNTY_CODE,COUNTY_NAME,JURISDICTION_CODE,JURISDICTION_NAME,SCHOOL_DISTRICT_CODE,SCHOOL_DISTRICT_NAME,ELECTION_DATE,IS_ABSENTEE_VOTER
0,102829192,63,OAKLAND,85480,WEST BLOOMFIELD TOWNSHIP,6090,BLOOMFIELD HILLS SCHOOLS,2018-11-06,N
1,102576042,47,LIVINGSTON,60120,OCEOLA TOWNSHIP,17910,HARTLAND CONSOLIDATED SCHOOLS,2012-11-06,N
2,102028985,61,MUSKEGON,19660,DALTON TOWNSHIP,36300,WHITEHALL DISTRICT SCHOOLS,2018-11-06,N
3,33728636,28,GRAND TRAVERSE,52480,MAYFIELD TOWNSHIP,20380,KINGSLEY AREA SCHOOLS,2020-03-10,N
4,32516942,58,MONROE,66900,RAISINVILLE TOWNSHIP,19050,IDA PUBLIC SCHOOLS,2012-08-07,N


In [68]:
recent_hist_df = voter_hist_df[voter_hist_df['ELECTION_DATE'] > datetime.date(2020, 1, 1)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [70]:
recent_hist_df.head(10)

,VOTER_IDENTIFICATION_NUMBER,COUNTY_CODE,COUNTY_NAME,JURISDICTION_CODE,JURISDICTION_NAME,SCHOOL_DISTRICT_CODE,SCHOOL_DISTRICT_NAME,ELECTION_DATE,IS_ABSENTEE_VOTER
3,33728636,28,GRAND TRAVERSE,52480,MAYFIELD TOWNSHIP,20380,KINGSLEY AREA SCHOOLS,2020-03-10,N
11,8800072,51,MANISTEE,50740,MANISTEE TOWNSHIP,22410,MANISTEE AREA SCHOOLS,2020-08-04,Y
28,4909625,25,GENESEE,33300,GRAND BLANC TOWNSHIP,16350,GRAND BLANC COMMUNITY SCHOOLS,2020-08-04,Y
32,3816327,73,SAGINAW,70520,SAGINAW CITY,30390,SAGINAW CITY SCHOOL DISTRICT,2020-03-10,N
52,32529917,73,SAGINAW,70540,SAGINAW TOWNSHIP,30450,SAGINAW TOWNSHIP COMMUNITY SCHOOLS,2020-03-10,N
69,6374127,41,KENT,34000,GRAND RAPIDS CITY,16440,GRAND RAPIDS PUBLIC SCHOOLS,2020-03-10,Y
70,6215583,82,WAYNE,11220,BROWNSTOWN TOWNSHIP,36485,WOODHAVEN-BROWNSTOWN SCHOOL DISTRICT,2020-03-10,Y
72,5643430,41,KENT,59580,OAKFIELD TOWNSHIP,17160,GREENVILLE PUBLIC SCHOOLS,2020-08-04,Y
78,4470307,74,ST CLAIR,65820,PORT HURON CITY,28830,PORT HURON AREA SCHOOL DISTRICT,2020-08-04,Y
79,159965410,78,SHIAWASSEE,61940,OWOSSO CITY,27210,OWOSSO PUBLIC SCHOOLS,2020-08-04,Y


In [33]:
mi_voter_senior_df['VOTER_STATUS_TYPE_CODE'].value_counts()

A     9273
V     2105
CH    1152
Name: VOTER_STATUS_TYPE_CODE, dtype: int64

In [49]:
pd.set_option('display.max_rows', None)
mi_voter_100_yrs_old_df = mi_voter_senior_df[['FIRST_NAME', 'LAST_NAME',  'YEAR_OF_BIRTH', 'ZIP_CODE', 'COUNTY_NAME', 'VOTER_STATUS_TYPE_CODE',]].sort_values(['YEAR_OF_BIRTH', 'VOTER_STATUS_TYPE_CODE',], ascending= True)
print(mi_voter_100_yrs_old_df)

            FIRST_NAME              LAST_NAME  YEAR_OF_BIRTH  ZIP_CODE  \
6370058        CHARLES                 BUTLER           1800     49032   
7857870          LAURA                 CUTLER           1800     49032   
3477411         EVELYN                SCHREVE           1800     49093   
876733           MAMIE                  JONES           1823     48226   
35893            AMBER                 HUGHES           1850     48726   
104586          THOMAS               HALLAUER           1850     49777   
246090            MARY               LAMPINEN           1850     49905   
266298           DONNA             SUNDERLAND           1850     49905   
455928        VIRGINIA                  MORAN           1850     48471   
591083          EILEEN                MIGNONI           1850     48475   
794378           SCOTT                  PRICE           1850     49963   
808354             AMY              AIT-BELLA           1850     49963   
965189           TAMMY            VELL

In [50]:
mi_voter_100_yrs_old_df.to_csv(os.path.join(DATA_ROOT_PATH, 'mi_voter_100_yrs_old.csv'))

In [73]:
mi_voter_senior_df = mi_voter_senior_df.rename(columns={'COUNTY_NAME': 'COUNTY_NAME_VOTER'})

In [74]:
mi_voter_100_yrs_old_voting_df = mi_voter_senior_df.merge(recent_hist_df, on='VOTER_IDENTIFICATION_NUMBER',how='inner')

In [77]:
print(mi_voter_100_yrs_old_voting_df[['FIRST_NAME', 'LAST_NAME','COUNTY_NAME', 'YEAR_OF_BIRTH', 'ELECTION_DATE', 'IS_ABSENTEE_VOTER', 'VOTER_STATUS_TYPE_CODE', 'ZIP_CODE']].sort_values(['YEAR_OF_BIRTH',], ascending= True))


        FIRST_NAME     LAST_NAME     COUNTY_NAME  YEAR_OF_BIRTH ELECTION_DATE  \
695          JAMES       BRADLEY           WAYNE           1900    2020-08-04   
65           ALICE         SAWKA         GENESEE           1900    2020-08-04   
64           ALICE         SAWKA         GENESEE           1900    2020-03-10   
656           JUNE         AIKEN         JACKSON           1900    2020-05-05   
564         ELAINE       HOFFMAN           WAYNE           1903    2020-03-10   
76          RONALD         ALLEN           WAYNE           1904    2020-08-04   
612           JOSE     HERNANDEZ           WAYNE           1904    2020-08-04   
102        ADELINE       GOODSON           WAYNE           1909    2020-08-04   
175        MATILDA        FOSTER           WAYNE           1909    2020-03-10   
165        CLINTON      ANDERSON           WAYNE           1911    2020-03-10   
446        CANDICE         COOTE         GENESEE           1911    2020-08-04   
447        CANDICE         C

In [78]:
mi_voter_100_yrs_old_voting_df.to_csv(os.path.join(DATA_ROOT_PATH, 'mi_voter_100_yrs_old_voting.csv'))